# LangChain with OpenAI

### Installation

In [ ]:
!pip install qdrant_client langchain langchain_huggingface langchain_community langchain_qdrant pypdf openai

### Configuration

In [ ]:
import openai
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
qdrant_api_key = userdata.get('QDRANT_API_KEY')

llm_name = "gemini-1.5-flash"
qdrant_url = "https://0d263df7-e830-4b3f-a1b4........."
embed_model = HuggingFaceEmbeddings(model_name = "BAAI/bge-small-en-v1.5")

# Q/A Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter

prompt_str = """
Answer the user question briefly

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prompt_str)
question_fetcher = itemgetter("question")
llm = ChatOpenAI(model_name = llm_name, openai_api_key = openai.api_key, temperature=0)

chain = question_fetcher | prompt | llm | StrOutputParser()
query = "tell me about Lahore"
response = chain.invoke({"question": query})

print(response)

# Q/A Retriever

In [ ]:
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import openai

### Loading PDF

In [ ]:
loaders = PyPDFLoader("")
pages = loaders.load()
len(pages)

### Splitting Text

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150)

splits = text_splitter.split_documents(pages)
len(splits)

## Initializing Qdrant Vector Store

In [ ]:
qdrant = QdrantVectorStore.from_documents(
    splits,
    embed_model,
    url = qdrant_url,
    prefer_grpc = True,
    api_key = qdrant_api_key,
    collection_name = "test_collection"
)

### Formatting Documents

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Chain Implementation

In [ ]:
prompt_str = """
Answer the user question only on the following context:
{context}
Question: {question}
"""

_prompt = ChatPromptTemplate.from_template(prompt_str)
num_chunks = 3

retriever = qdrant.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": num_chunks})

chat_llm = ChatOpenAI(
    model_name = llm_name,
    temperature = 0,
    openai_api_key = openai.api_key)

query_fetcher = itemgetter("question")
setup = {"question": query_fetcher, "context": query_fetcher | retriever | format_docs}
_chain = (setup | _prompt | chat_llm)

In [ ]:
query = "What is the Master Degree Course in Computer Science?"
response = chain.invoke({"question": query})

In [ ]:
response
response.content

# Conversational Chain

In [ ]:
history = []

In [ ]:
prompt_str = """
Answer the user question only on the following context:
{context}
Question: {question}
conversation_history: {chat_history}
"""

prompt = ChatPromptTemplate.from_template(prompt_str)
question_fetcher = itemgetter("question")
history_fetcher = itemgetter("chat_history")

llm = ChatOpenAI(
    model_name = llm_name,
    temperature = 0,
    openai_api_key = openai.api_key)

setup = {"question": query_fetcher, "context": query_fetcher | retriever | format_docs}
chain = (setup | _prompt | chat_llm)
query = "user question:" + query
response = "ai_response:" + response
history.append((query, response))

In [ ]:
history